In [40]:
! pip install datasets transformers seqeval


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.1 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=7e58dd454b7e59dbf758fe8d5f620358b1fe2f399c62ee9d52a4add9746b7944
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5

In [ ]:
#Bert ADE NER evaluation


In [5]:
#import evaluation dataset from SMM4H
#import raw tweets data from dev
import pandas as pd

RawTweets_ColName = ['tweetid','tweettext']
RawTweets = pd.read_csv('https://raw.githubusercontent.com/FANMISUA/TweetAENormalization/main/ADENormalization/Dev/tweets.tsv',delimiter='\t',names=RawTweets_ColName)

SpansNorm_ColName = ['tweetid','ADE','start','end','adeterms','ptcode']

SpansNorm = pd.read_csv('https://raw.githubusercontent.com/FANMISUA/TweetAENormalization/main/ADENormalization/Dev/spans_norm.tsv',delimiter='\t',names=SpansNorm_ColName)


In [7]:
print(RawTweets.head())

print(SpansNorm.head())


                     tweetid  \
0  SMM4H2022ykI8vN7jZYnV57AM   
1  SMM4H2022uCZV2SRsCe4vzjFm   
2  SMM4H20229Aha6m4XERqYdFWf   
3  SMM4H2022UAvDTQWOIacvBkzp   
4  SMM4H2022qNHntuJnkevkahGr   

                                           tweettext  
0  @USER_________ i found the humira to fix all m...  
1  @USER__________ have to go to a doc now to see...  
2  06.30 day 14 Rivaroxaban diary. Thanks to para...  
3  rt @USER_______: my philly dr prescribed me tr...  
4  ciprofloxacin: how do you expect to sleep when...  
                     tweetid  ADE  start  end             adeterms    ptcode
0  SMM4H2022ykI8vN7jZYnV57AM  ADE    119  125               nerves  10029177
1  SMM4H2022ykI8vN7jZYnV57AM  ADE    126  139        muscle spasms  10028334
2  SMM4H2022uCZV2SRsCe4vzjFm  ADE     61   68              gaining  10047896
3  SMM4H2022uCZV2SRsCe4vzjFm  ADE     91  110  gain like 50 pounds  10047896
4  SMM4H20229Aha6m4XERqYdFWf  ADE    118  134     frontal headache  10019211


In [337]:
consolidated_dataset = {}

# for id, text in RawTweets.values:
#   # print(id,text)
#   for SpanNorms in SpansNorm.loc[SpansNorm['tweetid'] == id].values:
#     # print( SpanNorms[4])
#     # print(text)
#     if id in consolidated_dataset:
#         consolidated_dataset[id]["effect_indices_start"] = SpanNorms[2]
#         consolidated_dataset[id]["effect_indices_end"] = SpanNorms[3]
#         consolidated_dataset[id]["effect"] = SpanNorms[4]

#     else:
#       consolidated_dataset[id] = {
#           "text": text,
#           "effect_indices_start": SpanNorms[2],
#           "effect_indices_end": SpanNorms[3],
#           "effect":SpanNorms[4]  }

#   # break

for id, text in RawTweets.values:
  # print(id,text)
  for SpanNorms in SpansNorm.loc[SpansNorm['tweetid'] == id].values:
    # print( SpanNorms[4])
    # print(text)
    consolidated_dataset[id] = {
        "id": id,
        "text": text,
        "drug": SpanNorms[4] ,
        "drug_indices_start":int(SpanNorms[2]),
        "drug_indices_end":int(SpanNorms[3]),
        "effect_indices_start": int(SpanNorms[2]),
        "effect_indices_end": int(SpanNorms[3]),
        "effect":SpanNorms[4],
        "ptcode": SpanNorms[5]}


  # break
# consolidated_dataset

In [338]:
df = pd.DataFrame(list(consolidated_dataset.values()))
print(df.shape)
df.head()


(65, 9)


,id,text,drug,drug_indices_start,drug_indices_end,effect_indices_start,effect_indices_end,effect,ptcode
0,SMM4H2022ykI8vN7jZYnV57AM,@USER_________ i found the humira to fix all m...,muscle spasms,126,139,126,139,muscle spasms,10028334
1,SMM4H2022uCZV2SRsCe4vzjFm,@USER__________ have to go to a doc now to see...,gain like 50 pounds,91,110,91,110,gain like 50 pounds,10047896
2,SMM4H20229Aha6m4XERqYdFWf,06.30 day 14 Rivaroxaban diary. Thanks to para...,frontal headache,118,134,118,134,frontal headache,10019211
3,SMM4H2022UAvDTQWOIacvBkzp,rt @USER_______: my philly dr prescribed me tr...,shakes,123,129,123,129,shakes,10040528
4,SMM4H2022qNHntuJnkevkahGr,ciprofloxacin: how do you expect to sleep when...,stomach is a cement mixer,52,77,52,77,stomach is a cement mixer,10042113


In [109]:
from datasets import Dataset, ClassLabel, Sequence, load_dataset, load_metric

# save to JSON to then import into Dataset object
df.to_json("dataset.jsonl", orient="records", lines=True)
cons_dataset = load_dataset("json", data_files="dataset.jsonl")


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [110]:
cons_dataset


DatasetDict({
    train: Dataset({
        features: ['text', 'drug', 'drug_indices_start', 'drug_indices_end', 'effect_indices_start', 'effect_indices_end', 'effect'],
        num_rows: 65
    })
})

In [118]:
cons_dataset["train"][0]

cons_dataset["train"]["drug_indices_start"]

# generate_row_labels(list(cons_dataset["train"][0]), verbose=True)


[126,
 91,
 118,
 123,
 52,
 45,
 60,
 75,
 36,
 26,
 36,
 36,
 108,
 108,
 113,
 33,
 80,
 32,
 96,
 36,
 35,
 76,
 72,
 0,
 105,
 67,
 53,
 83,
 105,
 8,
 37,
 10,
 98,
 60,
 34,
 9,
 89,
 88,
 26,
 13,
 99,
 51,
 35,
 60,
 29,
 118,
 108,
 17,
 20,
 99,
 91,
 126,
 13,
 81,
 59,
 46,
 62,
 137,
 0,
 111,
 27,
 84,
 45,
 63,
 13]

In [114]:
def generate_row_labels(row, verbose=False):
    """ Given a row from the consolidated `Ade_corpus_v2_drug_ade_relation` dataset,
    generates BIO tags for drug and effect entities.

    """

    text = row["text"]

    labels = []
    label = "O"
    prefix = ""

    # while iterating through tokens, increment to traverse all drug and effect spans
    drug_index = 0
    effect_index = 0

    tokens = tokenizer(text, return_offsets_mapping=True)

    for n in range(len(tokens["input_ids"])):
        offset_start, offset_end = tokens["offset_mapping"][n]

        # should only happen for [CLS] and [SEP]
        if offset_end - offset_start == 0:
            labels.append(-100)
            continue

        if offset_start == row["drug_indices_start"][drug_index]:
            label = "DRUG"
            prefix = "B-"

        elif offset_start == row["effect_indices_start"][effect_index]:
            label = "EFFECT"
            prefix = "B-"

        labels.append(label_list.index(f"{prefix}{label}"))

        if offset_end == row["drug_indices_end"][drug_index]:
            label = "O"
            prefix = ""
            drug_index += 1

        elif offset_end == row["effect_indices_end"][effect_index]:
            label = "O"
            prefix = ""
            effect_index += 1

        # need to transition "inside" if we just entered an entity
        if prefix == "B-":
            prefix = "I-"

    if verbose:
        print(f"{row}\n")
        orig = tokenizer.convert_ids_to_tokens(tokens["input_ids"])
        for n in range(len(labels)):
            print(orig[n], labels[n])
    tokens["labels"] = labels

    return tokens

In [142]:
import datasets

metric = datasets.load_metric("seqeval")

true_labels = [['O','AE','AE'],['O','AE','AE']]
true_predictions = [['O','AE','O'],['O','AE','AE']]

# results = metric.compute(predictions=predictions, references=true_labels)
results = metric.compute(predictions=true_predictions, references=true_labels)
results

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: AE seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'E': {'precision': 0.5, 'recall': 0.5, 'f1': 0.5, 'number': 2},
 'overall_precision': 0.5,
 'overall_recall': 0.5,
 'overall_f1': 0.5,
 'overall_accuracy': 0.8333333333333334}

In [82]:
label_list = ['O', 'B-DRUG', 'I-DRUG', 'B-EFFECT', 'I-EFFECT']

custom_seq = Sequence(feature=ClassLabel(num_classes=5,
                                         names=label_list,
                                         names_file=None, id=None), length=-1, id=None)

cons_dataset["train"].features["ner_tags"] = custom_seq


In [143]:
def convert_to_bio_labels(labeled_sentence):
    words = []
    labels = []

    for word, label in labeled_sentence:
        word_tokens = word.split()  # Split multi-word tokens
        words.extend(word_tokens)

        # Create BIO labels for each word
        if len(word_tokens) == 1:
            labels.append(f"B-{label}")
        else:
            labels.append(f"B-{label}")
            labels.extend([f"I-{label}"] * (len(word_tokens) - 1))

    return words, labels

# Example labeled sentence: [("Ibuprofen", "DRUG"), ("is", "O"), ("used", "O"), ("for", "O"), ("pain", "EFFECT")]
labeled_sentence = [("Ibuprofen", "DRUG"), ("is", "O"), ("used", "O"), ("for", "O"), ("pain", "EFFECT")]

words, bio_labels = convert_to_bio_labels(labeled_sentence)

print("Words:", words)
print("BIO Labels:", bio_labels)


Words: ['Ibuprofen', 'is', 'used', 'for', 'pain']
BIO Labels: ['B-DRUG', 'B-O', 'B-O', 'B-O', 'B-EFFECT']


In [54]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


In [144]:
#predict the adverse event label
from transformers import (AutoModelForTokenClassification,
                          AutoTokenizer,
                          pipeline,
                          )


model_checkpoint = "jsylee/scibert_scivocab_uncased-finetuned-ner"
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=5,
                                                        id2label={0: 'O', 1: 'B-DRUG', 2: 'I-DRUG', 3: 'B-EFFECT', 4: 'I-EFFECT'},max_length= 512
                                                        )
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


model_pipeline = pipeline(task="ner", model=model, tokenizer=tokenizer)


No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


In [145]:
print("long sentence test:",model_pipeline(" Compositionality: Splitting words into subword units enables word embeddings to handle compositionality, where the meaning of a complex word or phrase can be inferred from the meanings of its constituent parts. thanks :) i so wanted venlafaxine to work cos at least it didn't increase my appetite like citalopram did."))


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


long sentence test: [{'entity': 'B-DRUG', 'score': 0.9982658, 'index': 44, 'word': 'ven', 'start': 234, 'end': 237}, {'entity': 'I-DRUG', 'score': 0.99789494, 'index': 45, 'word': '##la', 'start': 237, 'end': 239}, {'entity': 'I-DRUG', 'score': 0.99927026, 'index': 46, 'word': '##fa', 'start': 239, 'end': 241}, {'entity': 'I-DRUG', 'score': 0.99856794, 'index': 47, 'word': '##xin', 'start': 241, 'end': 244}, {'entity': 'I-DRUG', 'score': 0.9982498, 'index': 48, 'word': '##e', 'start': 244, 'end': 245}, {'entity': 'B-EFFECT', 'score': 0.42705956, 'index': 58, 'word': 'increase', 'start': 277, 'end': 285}, {'entity': 'I-EFFECT', 'score': 0.5558925, 'index': 60, 'word': 'appetite', 'start': 289, 'end': 297}, {'entity': 'B-DRUG', 'score': 0.98945504, 'index': 62, 'word': 'cit', 'start': 303, 'end': 306}, {'entity': 'I-DRUG', 'score': 0.9701127, 'index': 63, 'word': '##alo', 'start': 306, 'end': 309}, {'entity': 'I-DRUG', 'score': 0.9732002, 'index': 64, 'word': '##pr', 'start': 309, 'end':

In [210]:
def extract_effect(entity):
  effects = []
  current_effect = ''

  for item in entity:
      if item['entity'] in ['B-EFFECT', 'I-EFFECT']:
          if item['entity'] == 'B-EFFECT':
              if current_effect:  # save previous effect if there's any
                  effects.append(current_effect.replace('##', ''))
              current_effect = item['word']
          elif item['entity'] == 'I-EFFECT':
              current_effect += ' ' + item['word']

  if current_effect:  # save the last effect
      effects.append(current_effect.replace('##', ''))

  print("effect list",effects)
  return effects

In [ ]:
def extract_effect(entity):
  effects = []
  current_effect = ''

  for item in entity:
      if item['entity'] in ['B-EFFECT', 'I-EFFECT']:
          if item['entity'] == 'B-EFFECT':
              if current_effect:  # save previous effect if there's any
                  effects.append(current_effect.replace('##', ''))
              current_effect = item['word']
          elif item['entity'] == 'I-EFFECT':
              current_effect += ' ' + item['word']

  if current_effect:  # save the last effect
      effects.append(current_effect.replace('##', ''))

  print("effect list",effects)
  return effects

In [246]:
[{'entity_group': 'EFFECT',
  'score': 0.73922336,
  'word': 'muscle spasms',
  'start': 125,
  'end': 138}]

def extract_effectterms(entity):
  effects = []
  current_effect = ''

  for item in entity:
      if item['entity_group'] in ['EFFECT']:
        current_effect = item['word']

  if current_effect:  # save the last effect
      effects.append(current_effect.replace('##', ''))

  print("effect list",effects)
  return effects

print(extract_effectterms(model_pipeline(df['text'][0], aggregation_strategy="simple"))[0])

effect list ['muscle spasms']
muscle spasms


In [167]:
#convert text into word BIO ner list
def label_sentence(text, label_subtext):
  labels = ["O"] * len(text.split())  # Initialize labels with 'O' for each token

  if label_subtext in text:
      label_tokens = label_subtext.split()
      start_idx = text.index(label_subtext)
      end_idx = start_idx + len(label_subtext)

      for idx, token in enumerate(text.split()):
          if start_idx <= len(" ".join(text.split()[:idx+1])) <= end_idx:
              labels[idx] = "B-AE" if idx == 0 else "I-AE"

  return labels

# text = "Apple is a tech company headquartered in Cupertino."
# label_subtext = "tech company"
# label_sentence(text,label_subtext)

In [242]:
model_pipeline(df['text'][0], aggregation_strategy="simple")

[{'entity_group': 'EFFECT',
  'score': 0.73922336,
  'word': 'muscle spasms',
  'start': 125,
  'end': 138}]

In [254]:
predictAEList = ['muscle spasm','gain']
for index, row in df.iterrows():
  if index < 2:
    print(row['text'], row['effect'])
    # print(label_sentence(row['text'], row['effect']))
    true_label.append(label_sentence(row['text'], row['effect']))
    print("true:",row['effect'])
    # print(model_pipeline(row['text']))
    # print(model_pipeline(row['text'],aggregation_strategy="simple"))
    # predict = model_pipeline(row['text'],aggregation_strategy="simple")
    words = [item['word'] for item in model_pipeline(row['text'],aggregation_strategy="simple")]
    print("predict:",words[0])
    # print(label_sentence(row['text'], words[0]))
    predict_label.append(label_sentence(row['text'], words[0]))

    # print(extract_effect(model_pipeline(row['text'])))

    # print(extract_effect(model_pipeline(row['text'])))
    # print("")
  else:
    break

print(true_label)
print(predict_label)
results = metric.compute(predictions=predict_label, references=true_label)

print(results)



@USER_________ i found the humira to fix all my crohn's issues, but cause other issues. i went off it due to issues w nerves/muscle spasms muscle spasms
true: muscle spasms
predict: muscle spasms
@USER__________ have to go to a doc now to see why i'm still gaining. stupid paxil made me gain like 50 pounds ?? and now i have to lose it gain like 50 pounds
true: gain like 50 pounds
predict: paxil
[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-AE', 'I-AE'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-AE', 'I-AE', 'I-AE', 'I-AE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]
[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-AE', 'I-AE'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-AE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]
{'AE': {'precisi

KeyError: ignored

In [291]:
df['text'][57], df['id'][57]

("trying to distract with masterchef, twitter, texting &amp; candy crush but i'm anxious &amp; have stomach &amp; head pain #venlafaxine #withdrawal",
 'SMM4H20220mSmvlK5h4hXJNh9')

In [292]:
model_pipeline(df['text'][57],aggregation_strategy="simple")

[{'entity_group': 'DRUG',
  'score': 0.9184949,
  'word': 'masterchef',
  'start': 24,
  'end': 34},
 {'entity_group': 'EFFECT',
  'score': 0.7456929,
  'word': 'stomach',
  'start': 98,
  'end': 105},
 {'entity_group': 'EFFECT',
  'score': 0.8821468,
  'word': 'head pain',
  'start': 112,
  'end': 121},
 {'entity_group': 'DRUG',
  'score': 0.99281883,
  'word': 'venlafaxine',
  'start': 123,
  'end': 134}]

In [282]:
for index, row in df.iterrows():
  print(index,df['id'][index],row['effect'])



0 SMM4H2022ykI8vN7jZYnV57AM muscle spasms
1 SMM4H2022uCZV2SRsCe4vzjFm gain like 50 pounds
2 SMM4H20229Aha6m4XERqYdFWf frontal headache
3 SMM4H2022UAvDTQWOIacvBkzp shakes
4 SMM4H2022qNHntuJnkevkahGr stomach is a cement mixer
5 SMM4H2022a2hde3adQrDE2HIV passing out
6 SMM4H2022hceks3meUmDcJLy8 insulin blocker
7 SMM4H2022Obitap6ILHXjD5SH increase my appetite
8 SMM4H2022MzDdtDqSq1LhsEKO ruined my teeth
9 SMM4H20221fZluiIo0whXaJA6 coming off
10 SMM4H2022RKHfXCWIZXEAIC6c shit feeling
11 SMM4H2022FupUyp3V1qkdfaQW confused
12 SMM4H2022TuXafFONznuqO8aQ get out of bed
13 SMM4H2022yfpTy24FLthRUaay cant keep still
14 SMM4H2022Xd3yVjFU568MhuMv triggers my rapid cycling
15 SMM4H2022r2Rhx2VTef0t53fP pins &amp; needles
16 SMM4H2022qpOud7DsjWMhWhFb rash
17 SMM4H2022w1Dip50oWfNYBVdo brain turn to mush
18 SMM4H2022Ktwet43odUtXgxNZ binge eat
19 SMM4H2022SDuYUDOiLTAqQMki nosleep
20 SMM4H2022UGqZXi74Aq6rkF4J murdering my mind
21 SMM4H2022dUpa56FmoF70Koge descent into madness
22 SMM4H2022eUHEdYa9HaG1N9d7 sick

In [304]:
#evaluate adverse event extraction
predictList = []
for index, row in df.iterrows():
  predictList.append(row['effect'])
# print(predictList[1])
i = [1,5,9,20,22,24,27,33,40,42,44,45,54,63]
for j in i:
  if j >=0 and j < len(predictList):
    predictList[j] = ''
predictList[4] = 'sleep'
predictList[8] = 'ruined'
predictList[11] = 'dizzy'
predictList[12] = 'sleep'
predictList[13] = 'nausea'
predictList[39] = 'spoonie'
predictList[50] = 'drunk'
predictList[57] = 'stomach'
predictList[64] = 'weight'



predictAEList = predictList
print(df['effect'][13])
# print(predictList)

for index, row in df.iterrows():
  # if index <100:
    # print(row['text'])
  true_label.append(label_sentence(row['text'], row['effect']))
  # print("true:",row['effect'])

  # print("predict:",predictAEList[index])
  predict_label.append(label_sentence(row['text'], predictAEList[index]))

  # print(extract_effect(model_pipeline(row['text'])))

  # print(extract_effect(model_pipeline(row['text'])))
  # print("")
  # else:
  #   break

print(true_label)
print(predict_label)
results = metric.compute(predictions=predict_label, references=true_label)

print(results)


cant keep still
[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-AE', 'I-AE'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-AE', 'I-AE', 'I-AE', 'I-AE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-AE', 'I-AE'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-AE', 'I-AE', 'I-AE', 'I-AE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-AE', 'I-AE'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-AE', 'I-AE', 'I-AE', 'I-AE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '

In [ ]:
#evaluate pt terms normalization


In [257]:
predictAEList = ['muscle spasm','gain','']
for index, row in df.iterrows():
  if index < 2:
    print(row['text'])
    true_label.append(label_sentence(row['text'], row['effect']))
    print("true:",row['effect'])
    # print(model_pipeline(row['text']))
    # print(model_pipeline(row['text'],aggregation_strategy="simple"))
    # predict = model_pipeline(row['text'],aggregation_strategy="simple")
    # words = [item['word'] for item in model_pipeline(row['text'],aggregation_strategy="simple")]
    # print("predict:",words[0])
    # print(label_sentence(row['text'], words[0]))
    print("predict:",predictAEList[index])
    predict_label.append(label_sentence(row['text'], predictAEList[index]))

    # print(extract_effect(model_pipeline(row['text'])))

    # print(extract_effect(model_pipeline(row['text'])))
    # print("")
  else:
    break

print(true_label)
print(predict_label)
results = metric.compute(predictions=predict_label, references=true_label)

print(results)



@USER_________ i found the humira to fix all my crohn's issues, but cause other issues. i went off it due to issues w nerves/muscle spasms
true: muscle spasms
predict: muscle spasm
@USER__________ have to go to a doc now to see why i'm still gaining. stupid paxil made me gain like 50 pounds ?? and now i have to lose it
true: gain like 50 pounds
predict: gain
[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-AE', 'I-AE'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-AE', 'I-AE', 'I-AE', 'I-AE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-AE', 'I-AE'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-AE', 'I-AE', 'I-AE', 'I-AE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'

In [253]:

predict = []
true_label, predict_label = [], []
for index, row in df.iterrows():
  if index < 2:
    print(row['text'])
    print("true label:",row['effect'])

    # print(extract_effectterms(model_pipeline(row['text'], aggregation_strategy="simple")))
  else:
    break

@USER_________ i found the humira to fix all my crohn's issues, but cause other issues. i went off it due to issues w nerves/muscle spasms
true label: muscle spasms
@USER__________ have to go to a doc now to see why i'm still gaining. stupid paxil made me gain like 50 pounds ?? and now i have to lose it
true label: gain like 50 pounds


In [177]:
output = [
    {'entity': 'B-EFFECT', 'score': 0.7441542, 'index': 41, 'word': 'muscle', 'start': 125, 'end': 131},
    {'entity': 'I-EFFECT', 'score': 0.76999676, 'index': 42, 'word': 'spa', 'start': 132, 'end': 135},
    {'entity': 'I-EFFECT', 'score': 0.8916267, 'index': 43, 'word': '##sm', 'start': 135, 'end': 137},
    {'entity': 'I-EFFECT', 'score': 0.5511157, 'index': 44, 'word': '##s', 'start': 137, 'end': 138}
]

# Sort the output by 'start' to ensure correct ordering
output.sort(key=lambda x: x['start'])

# Extract the words corresponding to 'B-EFFECT' and 'I-EFFECT' entities
extracted_words = [entity['word'] for entity in output if entity['entity'] in ['B-EFFECT', 'I-EFFECT']]
# Join the extracted words to form the result sentence
result_sentence = ' '.join(extracted_words)
# result_sentence.replace('##', '')
result_sentence
print(result_sentence)  # Output: "muscle spasms"


muscle spa ##sm ##s


In [209]:
output = [
    {'entity': 'B-EFFECT', 'score': 0.7441542, 'index': 41, 'word': 'muscle', 'start': 125, 'end': 131},
    {'entity': 'I-EFFECT', 'score': 0.76999676, 'index': 42, 'word': 'spa', 'start': 132, 'end': 135},
    {'entity': 'I-EFFECT', 'score': 0.8916267, 'index': 43, 'word': '##sm', 'start': 135, 'end': 137},
    {'entity': 'I-EFFECT', 'score': 0.5511157, 'index': 44, 'word': '##s', 'start': 137, 'end': 138}
]

# Sort the output by 'start' to ensure correct ordering
output.sort(key=lambda x: x['start'])

# Extract the words corresponding to 'B-EFFECT' and 'I-EFFECT' entities and remove "##"
extracted_words = [entity['word'].replace('##', '') for entity in output if entity['entity'] in ['B-EFFECT', 'I-EFFECT']]

# Join the extracted words to form the result sentence
result_sentence = ' '.join(extracted_words)

print(result_sentence)  # Output: "muscle spasms"


muscle spa sm s


In [202]:
words = ['muscle', 'spa', '##sm', '##s']

# Remove '#' characters and join the words without spaces
result = ''
# Join the result words to form the phrase
for word in words:
  print('1',word)
  if word.startswith('##'):
    word.replace('##', '')
    # result = ''.join(word)
    print('2',word)
    result += word
  else:
    result +=word

  # result += '
    # print(result)


print(result)


# print(result_phrase)  # Output: "musclespasms"


1 muscle
1 spa
1 ##sm
2 ##sm
1 ##s
2 ##s
musclespa##sm##s


In [183]:
# words = ['muscle', 'spa', '##sm', '##s']

# # Remove '#' characters, join the words, and split on '##'
# result_words = ' '.join([word.replace('##', '') for word in words]).split()

# # Join the result words to form the phrase
# result_phrase = ' '.join(result_words)

# print(result_phrase)  # Output: "muscle spasms"


muscle spa sm s


In [154]:
for index, row in df.iterrows():
  print(row['text'], row['effect'])
  # print()
  # print(extract_effect(model_pipeline(row['text'])))
  # print("")
  break


@USER_________ i found the humira to fix all my crohn's issues, but cause other issues. i went off it due to issues w nerves/muscle spasms muscle spasms


In [498]:
from google.colab import drive

# Mount your Google Drive
drive.mount('/content/drive')
meddra_path = '/content/drive/MyDrive/Data/Dev/llt.asc'  # Replace 'your_file.asc' with the actual path to your file

class Meddra(object):
    '''Basic Meddra Entity object'''
    def __init__(self, ptid, lltid, text):
        self.ptid = ptid
        self.lltid = lltid
        self.text = text

# def get_meddra_dict(llt_file):
#     """load corpus data and write resolution files"""
#     pt_dict, llt_dict = {}, {}
#     for line in open(llt_file, 'r'):
#         elems = line.split("$")
#         if len(elems) > 2:
#             ptid, lltid, text = elems[2], elems[0], elems[1]
#             # print(ptid, lltid, text)
#             entry = Meddra(ptid, lltid, text)
#             if ptid == lltid:
#                 pt_dict[ptid] = entry
#             llt_dict[lltid] = entry
#     return pt_dict, llt_dict

def getMedDRA():
  pt_dict, llt_dict = {}, {}
  # llt_file = "https://raw.githubusercontent.com/FANMISUA/TweetAENormalization/main/ADENormalization/MedDRA/llt.asc"
  # pt_dict, llt_dict = get_meddra_dict(llt_file)
  # Read the file from google drive
  # with open(file_path, 'r') as file:
  #   content = file.read()
  for line in open(meddra_path, 'r'):
    elems = line.split("$")
    if len(elems) > 2:
      ptid, lltid, text = elems[2], elems[0], elems[1]
      # print(ptid, lltid, text)
      entry = Meddra(ptid, lltid, text)
      if ptid == lltid:
          pt_dict[ptid] = entry
      llt_dict[lltid] = entry
  return pt_dict, llt_dict
  #   elems = line.split("$")
  #       # if len(elems) > 2:
  #   ptid, lltid, text = elems[2], elems[0], elems[1]
  #   print(ptid, lltid, text
  # print(content)
  # return pt_dict, llt_dict


#get pt terms list
pt_dict, llt_dict = getMedDRA()
# print()
print(len(pt_dict),len(llt_dict))

print(pt_dict)



        # w+=1

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
23389 79507
{'10000002': <__main__.Meddra object at 0x7f73e63b9b10>, '10000005': <__main__.Meddra object at 0x7f73e63b9bd0>, '10000007': <__main__.Meddra object at 0x7f73e63b9150>, '10000009': <__main__.Meddra object at 0x7f73e63bae60>, '10000011': <__main__.Meddra object at 0x7f73e63b9900>, '10000013': <__main__.Meddra object at 0x7f73e63b9060>, '10000014': <__main__.Meddra object at 0x7f73e63b85b0>, '10000020': <__main__.Meddra object at 0x7f73e63bace0>, '10000021': <__main__.Meddra object at 0x7f73e63bb3d0>, '10000028': <__main__.Meddra object at 0x7f73e63ba3b0>, '10000029': <__main__.Meddra object at 0x7f73e63ba7a0>, '10000044': <__main__.Meddra object at 0x7f73e63ba4a0>, '10000050': <__main__.Meddra object at 0x7f73e63ba1a0>, '10000059': <__main__.Meddra object at 0x7f73e63bbc40>, '10000060': <__main__.Meddra object at 0x7f73e63ba140>, '10000077': <__mai

In [499]:
pt_terms = []
pt_ids = []
llt_terms = []
llt_ids = []

for key, value in llt_dict.items():
  llt_terms.append(value.text)
  llt_ids.append(int(value.lltid))



for key, value in pt_dict.items():
  # if w < 10:
  # print(key, value, value.text, value.ptid)
  pt_terms.append(value.text)
  pt_ids.append(int(value.ptid))

In [434]:
print(llt_ids)

[10001890, 10000002, 10033315, 10033315, 10000005, 10061608, 10000007, 10000009, 10000009, 10000007, 10000011, 10020539, 10000013, 10000014, 10021684, 10021684, 10021684, 10003674, 10046940, 10000020, 10000021, 10003677, 10046940, 10044592, 10044594, 10044596, 10046940, 10000028, 10000029, 10067528, 10068532, 10020916, 10069571, 10003193, 10001052, 10001562, 10001569, 10000081, 10000081, 10000060, 10000059, 10000059, 10000044, 10000060, 10000060, 10000647, 10060921, 10000621, 10000050, 10002882, 10002882, 10002886, 10002882, 10000081, 10000081, 10000081, 10000081, 10000059, 10000060, 10000060, 10000059, 10021118, 10000060, 10000090, 10060954, 10060954, 10060922, 10060923, 10060954, 10060954, 10021151, 10060924, 10000077, 10000077, 10067715, 10000077, 10000081, 10000081, 10000081, 10000084, 10000081, 10000087, 10000087, 10014166, 10052489, 10000090, 10061936, 10061502, 10061937, 10000094, 10060926, 10000647, 10000097, 10000098, 10000099, 10000279, 10000101, 10004721, 10004722, 10004723,

In [435]:
all_integers = all(isinstance(item, int) for item in pt_ids)
all_strings = all(isinstance(item, str) for item in pt_ids)

if all_integers:
    print("All elements are integers")
elif all_strings:
    print("All elements are strings")
else:
    print("Mixed types")

All elements are integers


In [501]:
# testpt = "10028334"
testpt = "10047896"

# if testpt in pt_ids:
#   print('yes')
#   for key, value in pt_dict.items():
#     if value.ptid == testpt:
#       print(value.text)

if testpt in llt_ids:
  print('yes')
  for key, value in llt_dict.items():
    if int(value.lltid) == int(testpt):
      print(value.text)
print(testpt)

10047896


In [412]:
df.head()


,id,text,drug,drug_indices_start,drug_indices_end,effect_indices_start,effect_indices_end,effect,ptcode
0,SMM4H2022ykI8vN7jZYnV57AM,@USER_________ i found the humira to fix all m...,muscle spasms,126,139,126,139,muscle spasms,10028334
1,SMM4H2022uCZV2SRsCe4vzjFm,@USER__________ have to go to a doc now to see...,gain like 50 pounds,91,110,91,110,gain like 50 pounds,10047896
2,SMM4H20229Aha6m4XERqYdFWf,06.30 day 14 Rivaroxaban diary. Thanks to para...,frontal headache,118,134,118,134,frontal headache,10019211
3,SMM4H2022UAvDTQWOIacvBkzp,rt @USER_______: my philly dr prescribed me tr...,shakes,123,129,123,129,shakes,10040528
4,SMM4H2022qNHntuJnkevkahGr,ciprofloxacin: how do you expect to sleep when...,stomach is a cement mixer,52,77,52,77,stomach is a cement mixer,10042113


In [503]:
# print(df['ptcode'].shape)
TrueAENormPT = df[['effect','ptcode']]
PredictAENormPT = []
# print(len(pt_terms),len(pt_ids))
# print(TrueAENormPT.shape,TrueAENormPT.head())

# print(len(predictAEList))


for True_AEPTid in TrueAENormPT['ptcode']:
  # print(True_AEPTid)
  if True_AEPTid in llt_ids:
    print(True_AEPTid)
    # TrueAENormPT['ptcode_terms'] =

      # PredictAENormPT.append(value.text)

# print(PredictAENormPT)

print(TrueAENormPT.head())

#   for key, value in pt_dict.items():
#     if value.ptid == dfid:
#   # if dfid in llt_ids:
#       print(dfid)
#     # else:
#     #   print("not found",dfid)


#   # print(ptid)
#   # for key, value in pt_dict.items():
#   #   if dfid == int(value.ptid)
#   #     print(value.ptid,value.text)
#   #   # else:
#     #   print('not found',dfid)
#   # print(pt_dict.items[value].ptid)



10028334
10047896
10019211
10040528
10042113
10041349
10005613
10021654
10044034
10013754
10016370
10010300
10041349
10038743
10065032
10035067
10037844
10050462
10004716
10041017
10027374
10027352
10016365
10028822
10001125
10013632
10042494
10047896
10016384
10040995
10042661
10027374
10046317
10011865
10073281
10040528
10019304
10041014
10037234
10033371
10042661
10013932
10042494
10056484
10041014
10048010
10021654
10027175
10013767
10018732
10041053
10003988
10074314
10047896
10047896
10038742
10036661
10073281
10004063
10022989
10013781
10044698
10029898
10012336
10047896
                      effect    ptcode
0              muscle spasms  10028334
1        gain like 50 pounds  10047896
2           frontal headache  10019211
3                     shakes  10040528
4  stomach is a cement mixer  10042113


In [441]:
# print(df['ptcode'].shape)
TrueAENormPT = df[['effect','ptcode']]
PredictAENormPT = []
# print(len(pt_terms),len(pt_ids))
# print(TrueAENormPT.shape,TrueAENormPT.head())

# print(len(predictAEList))


for True_AEPTid in TrueAENormPT['ptcode']:
  # print(True_AEPTid)
  if True_AEPTid in pt_ids:
    print(True_AEPTid)
    # TrueAENormPT['ptcode_terms'] =

      # PredictAENormPT.append(value.text)

# print(PredictAENormPT)

print(TrueAENormPT.head())

#   for key, value in pt_dict.items():
#     if value.ptid == dfid:
#   # if dfid in llt_ids:
#       print(dfid)
#     # else:
#     #   print("not found",dfid)


#   # print(ptid)
#   # for key, value in pt_dict.items():
#   #   if dfid == int(value.ptid)
#   #     print(value.ptid,value.text)
#   #   # else:
#     #   print('not found',dfid)
#   # print(pt_dict.items[value].ptid)



10028334
10019211
10041349
10005613
10021654
10044034
10013754
10041349
10038743
10037844
10004716
10027374
10027374
10019304
10033371
10013932
10048010
10021654
10027175
10003988
10036661
10013781
10029898
                      effect    ptcode
0              muscle spasms  10028334
1        gain like 50 pounds  10047896
2           frontal headache  10019211
3                     shakes  10040528
4  stomach is a cement mixer  10042113


In [495]:
TrueAENormPT = {'effect':df['effect'],
                'ptcode':df['ptcode']}
TrueAENormPT = pd.DataFrame(TrueAENormPT)



# print("predict list",PredictAENormPT.head())



for index, row in TrueAENormPT.iterrows():
  # print(index,row['effect'])
  # print(index, row['ptcode'])
  for key, value in pt_dict.items():
    if int(value.ptid) == int(row['ptcode']):
      # print(value.ptid,value.text)
      TrueAENormPT.loc[index, 'ptcode_terms'] = value.text

print(TrueAENormPT)



# for index, True_AEPTid in TrueAENormPT['ptcode']:
#   print(True_AEPTid)
# #   if True_AEPTid in pt_ids:
# #     print(True_AEPTid)
#     # TrueAENormPT['ptcode_terms'] =

#       # PredictAENormPT.append(value.text)

# # print(PredictAENormPT)

# print(TrueAENormPT.head())





                       effect    ptcode                   ptcode_terms
0               muscle spasms  10028334                  Muscle spasms
1         gain like 50 pounds  10047896                            NaN
2            frontal headache  10019211                       Headache
3                      shakes  10040528                            NaN
4   stomach is a cement mixer  10042113                            NaN
..                        ...       ...                            ...
60               cotton mouth  10013781                      Dry mouth
61       couldn't fall asleep  10044698                            NaN
62                        OCD  10029898  Obsessive-compulsive disorder
63                   addicted  10012336                            NaN
64              gained 30 lbs  10047896                            NaN

[65 rows x 3 columns]


In [496]:
!pip install fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

PredictAENormPT = {'effect':predictAEList}

PredictAENormPT = pd.DataFrame(PredictAENormPT)

for index, row in PredictAENormPT.iterrows():
  print(index,row['effect'])
  best_match, score = process.extractOne(row['effect'], pt_terms, scorer=fuzz.partial_ratio)

  # print(best_match, score)
  if score >= 80:
    # print(best_match, score)
    for key, value in pt_dict.items():
      if value.text == best_match:
        # print(value.ptid,value.text)
        PredictAENormPT.loc[index, 'ptcode'] = value.ptid
    PredictAENormPT.loc[index, 'ptcode_terms'] = best_match

print(PredictAENormPT)



0 muscle spasms


1 
2 frontal headache
3 shakes
4 sleep


5 
6 insulin blocker
7 increase my appetite
8 ruined


9 
10 shit feeling
11 dizzy
12 sleep
13 nausea
14 triggers my rapid cycling
15 pins &amp; needles
16 rash
17 brain turn to mush
18 binge eat
19 nosleep


20 
21 descent into madness


...

40 
41 dyslexic


42 
43 craving


44 
45 
46 appetite
47 making me stupid
48 drunk
49 knackered when I wake
50 drunk
51 back pain
52 appetite
53 gain alot of weight


54 
55 restless legs
56 priapism
57 stomach
58 Horrendous venlafaxine dream
59 irrational
60 cotton mouth
61 couldn't fall asleep
62 OCD


63 
64 weight
                  effect    ptcode             ptcode_terms
0          muscle spasms  10028334            Muscle spasms
1                              NaN                      NaN
2       frontal headache  10019211                 Headache
3                 shakes  10048462      Growth of eyelashes
4                  sleep  10001423     Advanced sleep phase
..                   ...       ...                      ...
60          cotton mouth       NaN                      NaN
61  couldn't fall asleep  10016173                     Fall
62                   OCD       NaN                      NaN
63                             NaN                      NaN
64                weight  10000159  Abnormal loss of weight

[65 rows x 3 columns]


In [506]:
PredictAENormPT

[]

In [477]:
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

csv_file_path = '/content/drive/MyDrive/Data/Result/AENormTrue.csv'
TrueAENormPT.to_csv(csv_file_path, index=False)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [478]:
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

csv_file_path = '/content/drive/MyDrive/Data/Result/AENormPred.csv'
PredictAENormPT.to_csv(csv_file_path, index=False)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [505]:
print(PredictAENormPT)
# print(TrueAENormPT)



[]


In [493]:
MergeDataFrame = df[['id']]
# MergeDataFrame[]
for index, row in TrueAENormPT.iterrows():
    # print(index,row['effect'])
    print(PredictAENormPT['effect'][index],PredictAENormPT['ptcode_terms'][index])
    MergeDataFrame.loc[index, 'True_AE_Terms'] = row['effect']
    MergeDataFrame.loc[index, 'True_AE_PT_Code'] = row['ptcode']
    MergeDataFrame.loc[index, 'True_AE_PT_Terms'] = row['ptcode_terms']
    MergeDataFrame.loc[index, 'Predict_AE_Terms'] = PredictAENormPT['effect'][index]
    MergeDataFrame.loc[index, 'Predict_AE_PT_Code'] = PredictAENormPT['ptcode'][index]
    MergeDataFrame.loc[index, 'Predict_AE_PT_Terms'] = PredictAENormPT['NormedPT'][index]






print(MergeDataFrame)





muscle spasms 10028334


<ipython-input-493-5927082a5af7>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MergeDataFrame.loc[index, 'True_AE_Terms'] = row['effect']
<ipython-input-493-5927082a5af7>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MergeDataFrame.loc[index, 'True_AE_PT_Code'] = row['ptcode']
<ipython-input-493-5927082a5af7>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https

KeyError: ignored